In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import json
from shapely.geometry import LineString, Polygon, Point
from shapely import wkt

from ETL import utils

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data2015 = pd.read_csv('MyLA311_Service_Request_Data_2015.csv')

Convert to datetime and fill in placeholders

In [3]:
utils.to_datetime(data2015)

In [4]:
utils.fill_placeholder_1900_col(data2015)

Add datediff columns

In [5]:
utils.add_datediff_cols(data2015)

In [21]:
# utils.to_geom(data2015) - not needed - can just use geodf

In [35]:
gdf = gp.GeoDataFrame(
    data2015, geometry=gp.points_from_xy(data2015.Longitude, data2015.Latitude),crs={'init':'epsg:4326'})

In [32]:
len(gdf.ServiceDate.index) - len(gdf.ServiceDate.dropna().index)

11118

In [36]:
print(gdf.crs)

{'init': 'epsg:4326'}


In [120]:
gdf.geometry.distance(gdf.geometry.iloc[0]).sort_values().values < 1E-3

array([ True,  True,  True, ..., False, False, False])

What do we want?
- step by each row
    - use create and closed/service dates to get a subdf
        - if no closed date, consider all records from CreatedDate onwards
    - location match within subdf
    - requesttype match within subdf
    - check how far apart closeddates are to matches

In [138]:
def find_potential_matches(row,df):
    
    if pd.isnull(row.ClosedDate):
        closed = pd.to_datetime('2019-09-01')
    else:
        closed = row.ClosedDate
    
    return df.query(f'"{row.CreatedDate}" < CreatedDate < "{closed}" & \
                      RequestType == "{row.RequestType}"')
    
    #return df[(df.CreatedDate > row.CreatedDate) & (df.CreatedDate < closed)]

def loc_filter(loc,subdf):
    return subdf[subdf.distance(loc).values < 1E-3]

In [139]:
for index,row in gdf.head(1000).iterrows():
    match_candidates = find_potential_matches(row,gdf)
    if len(match_candidates.index):
        matches = loc_filter(row.geometry,match_candidates)
        
        if len(matches.index) > 1:
            print(row.Address, matches.Address)
    
    #print(len(match_reqtypes(row,gdf).index))

7743 N MILWOOD AVE, 91304 70     7729 N MILWOOD AVE, 91304
322    7743 N MILWOOD AVE, 91304
Name: Address, dtype: object
1836 N NEW HAMPSHIRE AVE, 90027 20    1836 N NEW HAMPSHIRE AVE, 90027
21    1836 N NEW HAMPSHIRE AVE, 90027
Name: Address, dtype: object
5308 N HARMONY AVE, 91601 805    5250 N HARMONY AVE, 91601
810    5250 N HARMONY AVE, 91601
Name: Address, dtype: object
150 W 93RD ST, 90003 379        162 W 92ND ST, 90003
815    150 1/2 W 93RD ST, 90003
Name: Address, dtype: object
1666 E 111TH PL, 90059 409    1660 E 111TH PL, 90059
834    1666 E 111TH PL, 90059
Name: Address, dtype: object


In [111]:
gdf.head().ClosedDate

0   2016-01-02 14:29:52
1                   NaT
2   2016-01-06 09:38:51
3   2016-01-02 09:04:07
4   2016-01-02 09:02:11
Name: ClosedDate, dtype: datetime64[ns]

In [134]:
gdf.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'MobileOS', 'Anonymous',
       'AssignTo', 'ServiceDate', 'ClosedDate', 'AddressVerified',
       'ApproximateAddress', 'Address', 'HouseNumber', 'Direction',
       'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude', 'Location',
       'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember', 'NC',
       'NCName', 'PolicePrecinct', 'ClosedDiff', 'ServiceDiff',
       'ClosedServiceDiff', 'ClosedDiff_Days', 'ServiceDiff_Days',
       'ClosedServiceDiff_Days', 'geometry'],
      dtype='object')

'5 days 03:57:10'